In [ ]:
import functions as fc
import pandas as pd

In [ ]:
fc.download_tgz_raw()

In [ ]:
fc.import_tgz("2018", "./data2018")

In [ ]:
fc.import_tgz("2019", "./data2019")

In [ ]:
fc.import_tgz("2020", "./data2020")

In [ ]:
df_final = fc.transform_rcs("2018")

In [ ]:
df_final.head()

In [ ]:
fc.export_2_minio("2018", df_final)

In [ ]:
fc.export_2_minio("2019", df_final)

In [ ]:
fc.export_2_minio("2020", df_final)

In [ ]:
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import requests
from zipfile import ZipFile

def isDirectory(url):
    if(url.endswith('/') and ".." not in url):
        return True
    else:
        return False
list_urls = []
def findLinks(url):
    page = requests.get(url).content
    bsObj = BeautifulSoup(page, 'html.parser')
    maybe_directories = bsObj.findAll('a', href=True)
    list_df = []
    
    for link in maybe_directories:
        #print(link['href'])
        # print(isDirectory(link['href']))
        if(isDirectory(link['href'])):
            newUrl = url + link['href']   
            print(newUrl)   
            list_urls.append(newUrl)  
            print(list_urls) 
            findLinks(newUrl) #recursion happening here$
            
            
        else:
            if(link['href'].endswith('.zip')):
                print("GOTCHA!") #now safe and download
                target_path = link['href']
                response = requests.get(list_urls[-1] + link['href'], stream=True)
                if response.status_code == 200:
                    with open(target_path, 'wb') as f:
                        f.write(response.raw.read())
                
                # zip_file = ZipFile(list_urls[-1] + link['href'])         
                # temp_df = [open_complex_file(zip_file, text_file.filename)
                # for text_file in zip_file.infolist()
                # if text_file.filename.endswith('.csv')]
                # list_df.append(temp_df)

startUrl = "http://data.cquest.org/inpi_rncs/imr/stock/2017/"
findLinks(startUrl)

In [ ]:
def transform_rcs(year, path2data=os.getcwd()):
    if year in ["2019", "2020"]:
        list_df = fc.import_all_files("data" + year + "/", "*.zip")
    if year in ["2017"]:
        list_df = fc.import_all_files(path2data + "/", "*.zip")
    if year in ["2018"]:
        list_df = fc.import_all_files("data" + year + "/", "*.csv")
    print(len(list_df))
    if year in ["2017","2019", "2020"]:
        li = []
        for i in range(len(list_df)):
            temp = pd.concat(list_df[i])
            li.append(temp)
        df_final = pd.concat(li)
    if year in ["2018"]:
        df_final = pd.concat(list_df)
    print(df_final.shape)
    return df_final

In [ ]:
del(list_df)

In [ ]:
import os
path2data=os.getcwd()

In [ ]:
path2data

In [ ]:
import functions as fc
list_df = fc.import_all_files(path2data + "/", "*.zip")

In [ ]:
len(list_df)

In [ ]:
sys.getsizeof(list_df)

In [ ]:
len(list_df[79])

In [ ]:
list_df[79]

In [ ]:
import pandas as pd
temp = pd.concat(list_df[0])

In [ ]:
temp.head()

In [ ]:
temp.shape

In [ ]:
def export_2_minio(year, path2output, df_final):
    S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
    BUCKET_OUT = "radjerad"
    FILE_KEY_OUT_S3 = "inpi/" + year + "/" + path2output 
    FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

    with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
        df_final.to_csv(file_out)
    return None


In [ ]:
import pandas as pd
li = []
import s3fs
for i in range(len(list_df)):
    print(i)
    temp = pd.concat(list_df[i])
    print(temp.shape)
    temp = temp.reset_index()
    path2output = temp.loc[0,"source"][0:16]
    print(path2output)
    #temp.to_csv(path2output, sep=";")
    export_2_minio("2017", path2output + ".csv", temp)
    # temp.to_csv(path2output + ".csv", sep=";")
    # li.append(temp)


In [ ]:
import os

import s3fs
import pandas as pd
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "radjerad/inpi/2017"
list_inpi_files = fs.ls(BUCKET)


In [ ]:
list_inpi_files = [f for f in list_inpi_files if ".csv" in f]

In [ ]:
list_inpi_files

In [ ]:
li = []
for i in list_inpi_files[0:42] :
    FILE_PATH_S3 = i
    with fs.open(FILE_PATH_S3, mode="rb") as file_in:
        print(i)
        temp = pd.read_csv(file_in, sep=",")
        temp = temp.drop(["index", "Unnamed: 0"], axis=1)
        li.append(temp)

In [ ]:
df_final = pd.concat(li)